In [8]:
import time
import requests
from bs4 import BeautifulSoup
import sqlite3

In [12]:
def scraping_and_saving(my_github_id, db_file_name):
    # --- 1. スクレイピング---
    
    # 自分のリポジトリ一覧ページのURLを作る
    url = f"https://github.com/{my_github_id}?tab=repositories"
    headers = {'User-Agent': 'Mozilla/5.0'}
    
    print(f"-> {url} にアクセス")
    
    time.sleep(1) 
    
    try:
        # 実際にWebページを取得
        res = requests.get(url, headers=headers)
        res.raise_for_status() # 失敗したらエラーに
    except requests.exceptions.RequestException as e:
        print(f"Webページ取得エラー: {e}")
        return

    # 取得したHTMLを解析できるようにする
    soup = BeautifulSoup(res.text, 'html.parser')
    
    # リポジトリの情報が入ってる場所（HTMLのパーツ）を全部見つける
    repo_list_items = soup.find('div', id='user-repositories-list').find_all('li')
    
    # DBに入れるためのデータを一時的に保存するリスト
    data_for_db = [] 
    
    print(f"リポジトリ {len(repo_list_items)} 個分を解析中…")
    
    for repo_item in repo_list_items:
        # リポジトリ名を探す
        name_tag = repo_item.find('h3', class_='wb-break-all').find('a')
        repo_name = name_tag.get_text(strip=True) if name_tag else 'N/A'
        
        # 主要な言語を探す
        lang_tag = repo_item.find('span', itemprop='programmingLanguage')
        language = lang_tag.get_text(strip=True) if lang_tag else 'N/A'
        
        # スターの数を探す
        star_tag = repo_item.find('a', href=lambda href: href and '/stargazers' in href)
        star_count = star_tag.get_text(strip=True).replace(',', '') if star_tag else '0'
        
        # 取得したデータをリストに追加
        data_for_db.append((repo_name, language, star_count))
        
        print(f"   - ゲット: {repo_name}, {language}, Star: {star_count}")
        time.sleep(1) 

    # --- 2. DBへの保存---
    
    db_path = '' 
    conn = None
    
    try:
        # DBに接続
        conn = sqlite3.connect(db_path + db_file_name)
        cur = conn.cursor()
        
        cur.execute("""
            CREATE TABLE IF NOT EXISTS repos (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                repo_name TEXT NOT NULL,
                language TEXT,
                stars TEXT
            )
        """)
        
        # 前回のデータは消しておく
        cur.execute("DELETE FROM repos")
        
        # SQL文の作成
        # '?' の部分に data_for_db の中身が入る
        sql = "INSERT INTO repos (repo_name, language, stars) VALUES (?, ?, ?)"
        
        # SQLの実行
        cur.executemany(sql, data_for_db)
        
        # 変更を確定
        conn.commit()
        print(f"保存完了: {len(data_for_db)}件のデータをDB ({db_file_name}) に保存")

    except sqlite3.Error as e:
        print(f'エラー: {e}')
        if conn:
            conn.rollback()
        
    finally:
        if conn:
            conn.close()
            print("DB接続終了")

In [13]:
def check_db_content(db_file_name):
    print("\n--- SELECT文でDBの中身をチェック ---")
    db_path = '' 
    conn = None
    
    try:
        # DBに接続
        conn = sqlite3.connect(db_path + db_file_name)
        # カーソルを取得
        cur = conn.cursor() 
        
        # SQL文の作成 
        sql = "SELECT id, repo_name, language, stars FROM repos"
        
        # SQLを実行
        cur.execute(sql)
        
        # 結果をまとめて取得
        results = cur.fetchall() 
        
        print("\n=== DBに保存されたデータ一覧（SELECT結果） ===")
        
        # ヘッダーを手動で表示
        print(f"{'ID':<4} | {'リポジトリ名':<30} | {'言語':<15} | {'Star':<5}")
        print("-----|--------------------------------|-----------------|------")
        
        # 取得した結果を一行ずつ表示
        for row in results:
            # 各列のデータを取り出す (row[0]はID, row[1]はリポジトリ名...)
            repo_id, name, lang, star = row 
            # <numberは表示幅の指定
            print(f"{repo_id:<4} | {name:<30} | {lang:<15} | {star:<5}")

        print("==============================================")
        
    except sqlite3.Error as e:
        print(f'エラー: {e}')
        
    finally:
        if conn:
            conn.close()
            print("DB接続終了")

In [14]:
MY_GITHUB_ID = "Nishiyama1204"
DB_NAME = "my_github_data.db"

scraping_and_saving(MY_GITHUB_ID, DB_NAME)
check_db_content(DB_NAME)

-> https://github.com/Nishiyama1204?tab=repositories にアクセス
リポジトリ 5 個分を解析中…
   - ゲット: dsprog_homework_1125, N/A, Star: 0
   - ゲット: dsprog2_2025, Python, Star: 0
   - ゲット: saisyu-kadai, Jupyter Notebook, Star: 0
   - ゲット: git_practice_2025, N/A, Star: 0
   - ゲット: dsprog1_2025, Jupyter Notebook, Star: 0
保存完了: 5件のデータをDB (my_github_data.db) に保存
DB接続終了

--- SELECT文でDBの中身をチェック ---

=== DBに保存されたデータ一覧（SELECT結果） ===
ID   | リポジトリ名                         | 言語              | Star 
-----|--------------------------------|-----------------|------
16   | dsprog_homework_1125           | N/A             | 0    
17   | dsprog2_2025                   | Python          | 0    
18   | saisyu-kadai                   | Jupyter Notebook | 0    
19   | git_practice_2025              | N/A             | 0    
20   | dsprog1_2025                   | Jupyter Notebook | 0    
DB接続終了
